In [3]:
using SparseArrays, LinearAlgebra

function readgraph(graph)
    I=Int[]
    J=Int[]
    open(graph) do f
        linecounter = 0
        for l in eachline(f)
            v = split(l)
            if v[1] == "D"
                i,j = extrema((parse(Int, v[2])+1, parse(Int, v[3])+1))
                push!(I, i)
                push!(J, j)
            end
        end
    end
    N = maximum(I)
    H = sparse(J.-N,I,fill(1,length(I))) .÷ 2
end


function readseeds(seeds, H)
    M, N = size(H)
    @show size(H)
    v = Int[]
    open(seeds) do f
        for l in eachline(f)
            if l[1] != '#'
                v = parse.(Int, split(l)) .- N .+ 1
            end
        end
        println("$(length(v)) removed factors $v")
        return H[setdiff(1:size(H,1), v), :]
    end
end

readseeds (generic function with 1 method)

In [4]:
function leaf_removal(H::SparseMatrixCSC)
    M, N = size(H)
    Ht = sparse(transpose(H)) # for faster access by row
    degs = vec(sum(H .!= 0, dims=1))
    facts = fill(true, M)
    rowperm = Int[]
    Q = findall(degs .== 1)
    indep = findall(degs .== 0)
    dep = Int[]
    while !isempty(Q)
        i = popfirst!(Q)
        degs[i] == 0 && continue
        push!(dep, i)
        ∂i = rowvals(H)[nzrange(H,i)]
        ∂i = ∂i[facts[∂i]]
        @assert length(∂i) == 1 # should be a residual leaf
        a = ∂i[1]
        facts[a] = false
        push!(rowperm, a) 
        for j in rowvals(Ht)[nzrange(Ht,a)]
            degs[j] -= 1
            if j != i
                if degs[j] == 0
                    push!(indep, j)
                elseif degs[j] == 1
                    push!(Q, j)
                end
            end
        end
    end
    all(degs .==  0) || @warn "non-empty core"
    rowperm, [dep; indep]
end

leaf_removal (generic function with 1 method)

In [5]:
isuppertriang(H::SparseMatrixCSC) = all(rowvals(H)[last(nzrange(H,i))] == i for i = 1:size(H,1))

function ut2diagGF2!(T::SparseMatrixCSC)
    (m,n) = size(T)
    # Check that the left part of T is unit upper triangular
    @assert isuppertriang(T)
    # Loop over diagonal elements
    for c in m:-1:2
        # Find non-zero elements above T[c,c] and perform row operations to 
        #  cancel them out
        for (j,v) in @views zip(rowvals(T[:,c]),nonzeros(T[:,c]))
            if v != 0 && j < c
                for k in rowvals(T[c,c:end]).+c.-1
                    T[j,k] ⊻= T[c,k]
                end
            end
        end
    end
    dropzeros!(T)
end

ut2diagGF2! (generic function with 1 method)

In [6]:
function findbasis(H)
    rowperm, colperm = leaf_removal(H)
    Hnew = H[rowperm, colperm]
    ut2diagGF2!(Hnew)
    B = [Hnew[:, size(Hnew, 1)+1:end]; I]
    indep = colperm[size(H,1)+1:end]
    B[invperm(colperm),:], indep
end 

findbasis (generic function with 1 method)

In [7]:
H = readgraph("/home/ab/src/spread/graphN9000.txt")
H = readseeds("/home/ab/src/spread/seedsN9000.txt", H)
B, indep = findbasis(H);
M,N = size(H)
α=1-M/N

size(H) = (6600, 9000)
5 removed factors [129, 1515, 1531, 1655, 3471]


0.26722222222222225

In [10]:
rowperm,colperm = leaf_removal(H)

([4568, 97, 303, 4962, 589, 3783, 2642, 554, 3162, 1042  …  1081, 4040, 5522, 198, 5901, 3075, 7, 3606, 4236, 3265], [91, 353, 364, 685, 3356, 3896, 4261, 4784, 4976, 5077  …  5628, 7363, 4170, 8751, 4673, 5935, 798, 7435, 4491, 8359])

In [11]:
H[rowperm, colperm]

6595×9000 SparseMatrixCSC{Int64, Int64} with 19783 stored entries:
⠙⢮⣖⣂⡇⡐⠂⣆⢥⣰⣤⣚⡴⠗⢭⢰⣝⠰⣷⠢⡩⣓⣂⢎⢓⡧⣵⢷⡆⣿⢍⡦⣘⣀⣧⢄⡲⢦⡲⠐
⠀⠀⠙⢿⡿⣝⢲⢮⢮⣧⢛⣮⡧⡷⣽⡀⡡⠮⡒⠮⣜⣓⠛⡹⡿⠕⢯⢭⠳⣿⡐⣡⡾⠟⠯⢓⢈⡢⠽⢆
⠀⠀⠀⠀⠙⢮⡙⢿⣻⠕⢙⣼⠪⣨⡞⡙⣴⡳⣹⡻⡢⡾⢚⢁⡵⡿⢸⡗⢫⣧⠶⡅⠏⣇⡦⣌⠳⠹⠐⣨
⠀⠀⠀⠀⠀⠀⠙⢦⡈⠛⢯⣧⠫⠹⢳⡻⢏⣓⠴⣋⣼⣗⣤⣶⡘⢧⣩⡭⠾⣿⢊⠹⣽⡻⠆⢣⠹⠦⡶⣔
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠻⢷⡗⣟⣆⡓⣧⢵⡓⣎⠋⡬⣭⡰⡥⡥⣼⣹⣷⠐⢘⣫⣟⡊⣰⣃⠿⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠙⠿⣮⠽⡿⠼⢌⢈⣇⡓⢹⡻⣷⠹⡻⠺⣿⡵⣫⣗⣣⡱⡳⢧⢑⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠳⣕⣹⣵⣍⠹⣨⢷⢱⡀⣣⠧⠀⣿⣿⣯⠵⣡⣾⡵⢔⠷⡶
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢮⣾⣻⠙⡾⣶⢧⡮⣨⡂⠸⣿⣿⣹⣋⣯⣻⣍⣨⠽
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢷⡜⣩⠆⡹⡼⠆⠀⠀⢿⣿⣧⡳⣵⡬⢍⠍⡛
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢶⡯⠿⣺⣐⡁⠀⠈⣿⣿⣿⣻⠗⠶⣽⠭
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⣧⡻⠷⠂⠀⠀⠘⣿⣿⣿⣧⣏⡦⢟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⢷⢗⡀⠀⠀⠀⠘⢿⣿⣿⣮⢷⡟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⢧⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⠋
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡘⡇⠀⠀⠀⠀⠀⠀⠘⠿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠃⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [361]:
using JLD
save("/home/ab/src/sourcecoding/matrix-stefano2.jld", "H", H)

In [11]:
include("/home/ab/src/sourcecoding/code/headers.jl")

In [51]:
ms = MS(maxiter=10000, convergence=:decvars, Tmax=1, gamma=1e-3, sigma=1e-8)
M,N = size(H)
@show 1-M/N
fg = FactorGraphGF2(H)
ProgressMeter.ijulia_behavior(:clear)

1 - M / N = 0.24002849002849003


false

In [119]:
using SparseArrays, Random

struct SurveyPropagation{F,M}
    H :: SparseMatrixCSC{F,Int}
    X :: SparseMatrixCSC{Int,Int}
    P :: Vector{M}
    Q :: Vector{M}
    belief :: Vector{M}
    efield :: Vector{Int}
    y :: Float64
end

function survey_propagation(H; field, init, y)
    H = sparse(H)
    X = sparse(SparseMatrixCSC(size(H)...,H.colptr,H.rowval,collect(1:length(H.nzval)))')
    P = [copy(init) for i=1:length(H.nzval)]
    Q = [copy(init) for i=1:length(H.nzval)]
    belief = [copy(init) for i=1:size(H,2)]
    SurveyPropagation(H, X, P, Q, belief, field, y)
end


function update_factor!(ms::SurveyPropagation, b)
    ε = 0.0
    ∂b = nzrange(ms.H, b)
    for i ∈ ∂b
        a = fill(0.0, -J-1:J+1)
        a[1:J] .= 1
        pi = 1.0
        for j ∈ ∂b
            i == j && continue
            q = ms.Q[j]
            Σp, Σm = 0.0, 0.0
            for f=J:-1:1
                ap, am = a[f], a[-f]
                Σp += q[f]; Σm += q[-f]
                a[+f] = ap*Σp + am*Σm
                a[-f] = am*Σp + ap*Σm
            end
            pi *= 1-q[0]
        end
        p = zero(ms.P[i])
        for f = 1:J
            p[+f] = (a[+f]-a[+f+1])
            p[-f] = (a[-f]-a[-f-1])*exp(2ms.y*f)
        end
        p[0] = 1-pi
        p ./= sum(p)
        ε = max(ε, maximum(abs, ms.P[i] - p))
        ms.P[i] .= p
    end
    ε
end


function ⊛(p1, p2)
    q = fill(0.0,firstindex(p1)+firstindex(p2):lastindex(p1)+lastindex(p2))
    for f1 in eachindex(p1), f2 in eachindex(p2)
        q[f1+f2] += p1[f1]*p2[f2]
    end
    q
end

function update_var!(ms::SurveyPropagation, i)
    ε = 0.0
    ∂i = @view nonzeros(ms.X)[nzrange(ms.H, i)]
    for a ∈ ∂i
        q1 = fill(1.0, ms.efield[i]:ms.efield[i])
        for b ∈ ∂i
            b == a && continue
            q1 = q1 ⊛ (ms.P[b] .* exp.(ms.y .* abs.(eachindex(ms.P[b]))))
        end
        q1 .*= exp.(-ms.y .* abs.(eachindex(q1)))
        q = zero(ms.Q[a])
        for f in eachindex(q1)
            q[clamp(f,-J,J)] += q1[f]
        end
        q ./= sum(q)
        ε = max(ε, maximum(abs, ms.Q[a] - q))
        ms.Q[a] .= q
    end
    ε
end

function iteration!(ms::SurveyPropagation; maxiter = 1000, tol=1e-3, γ=0.0, callback=(x...)->false)
    @inbounds for t = 1:maxiter        
        ε = max(maximum(update_factor!.(tuple(ms), 1:size(H,1))),
                maximum(update_var!.(tuple(ms), 1:size(H,2))))
        ε < tol && break
        callback(t, ε, ms) && break
    end
end                 


iteration! (generic function with 1 method)

In [120]:
using StaticArrays, OffsetArrays
s = rand((-1,1), size(H,2))
J = 5
sp = survey_propagation(H; field=s, init=OffsetArray(MVector{2J+1}(fill(1/(2J+1),2J+1)), -J:J), y=0.0);

In [123]:
callback(t, ε, sp) = (println("$t $ε"); false)

iteration!(sp, maxiter=20, callback=callback, tol=0)

1 0.0489358050587283
2 0.024760781918541475
3 0.026218370576922362
4 0.022477183328205474
5 0.016531183515554937
6 0.015406751446887856
7 0.015406751446887856
8 0.008595347164552597
9 0.00556228387766311
10 0.004382045734369289
11 0.003420336192796025
12 0.003420336192796025
13 0.002165813695206076
14 0.002165813695206076
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0


In [124]:
sp.Q

19783-element Vector{OffsetVector{Float64, Vector{Float64}}}:
 [0.0, 0.1575930600234846, 0.06618309413548393, 0.07126503426746286, 0.09090909090909088, 0.09090909090909094, 0.09090909090909094, 0.09090909090909088, 0.09090909090909088, 0.06172490702636202, 0.188688450001752]
 [0.12396694214876035, 0.04958677685950414, 0.057851239669421496, 0.06611570247933886, 0.07438016528925621, 0.08264462809917357, 0.09090909090909093, 0.08264462809917357, 0.07438016528925621, 0.06611570247933886, 0.23140495867768598]
 [0.15143650148089738, 0.04908383058782622, 0.057348293397743576, 0.06389901335601768, 0.09090909090909093, 0.08397339973052016, 0.08038447448323825, 0.07610011735413007, 0.07106444542412033, 0.06496078881844884, 0.21084004445796678]
 [0.12396694214876035, 0.04958677685950414, 0.057851239669421496, 0.06611570247933886, 0.07438016528925621, 0.08264462809917357, 0.09090909090909093, 0.08264462809917357, 0.07438016528925621, 0.06611570247933886, 0.23140495867768598]
 [0.557121657114827, 0

In [365]:
using ProgressMeter
callback(t,ms) = nothing;#println("$t $(ms.belief[1:4])")

nsamples = 10
ms = maxsum(H,  1 .- 2y .+ 1e-8*randn(N));

# the mapping is (0,1)=>(1,-1)
dist = map(1:nsamples) do i
    y = rand((0,1), N)
    ms.M .= 0
    ms.efield .= 1 .- 2y .+ 1e-8*randn(N)
    iteration!(ms; maxiter=2000, γ=1e-3, callback=callback)
    x = Int.(ms.belief .< 0)
    c = sum(ms.H'x .% 2)
    d = sum(x .⊻ y)/N
    @show d c
    x1 = B*x[indep] .% 2; d1 = sum(x1 .⊻ y)/N; c1 = sum(ms.H'x1 .% 2); @show d1 c1 
    d1
end
sum(dist)/nsamples

d = 0.26323218066337334
c = 0
d1 = 0.26323218066337334
c1 = 0
d = 0.2633498000470478
c = 2
d1 = 0.2635262291225594
c1 = 0
d = 0.26329099035521053
c = 0
d1 = 0.26329099035521053
c1 = 0
d = 0.26329099035521053
c = 5
d1 = 0.2646436132674665
c1 = 0
d = 0.2642907551164432
c = 5
d1 = 0.26523171018583863
c1 = 0
d = 0.265055281110327
c = 0
d1 = 0.265055281110327
c1 = 0
d = 0.2627028934368384
c = 1
d1 = 0.26299694189602446
c1 = 0
d = 0.26523171018583863
c = 0
d1 = 0.26523171018583863
c1 = 0
d = 0.2646436132674665
c = 26
d1 = 0.27111267936956013
c1 = 0
d = 0.26534932956951307
c = 6
d1 = 0.26940719830628085
c1 = 0


0.26537285344624795

In [366]:
1-M/N,sum(dist)/nsamples

(0.20418725005880967, 0.26537285344624795)

In [283]:
x=[1:10;]; f=(x .% 2 .== 1);x[findall(f)]==x[f]

true

In [52]:
dist = map(1:10) do i
    Random.seed!(i)
    
    y = rand((0,1), N)

    lm = LossyModelGF2(fg, zeros(N), Inf, 1.0, y)
    
    function checker()
        v = (B*((fg.fields .< 0)[indep])) .% 2;
        dist = sum(v .⊻ y)/N
        check = sum(H*(fg.fields .< 0) .% 2)
        check, dist
    end

    independent = falses(N)
    independent[indep] .= true
    res_MS = solve!(lm, ms, randseed=1, verbose=true, showprogress=true, 
        independent=independent, basis = B)
#    res_MS = solve!(lm, ms, randseed=1, verbose=true, showprogress=true)
    avg_dist = res_MS.distortion 
    check, dist = checker()
    @show dist check avg_dist
    dist
end

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24287749287749288
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24786324786324787
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


...

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.245014245014245
check = 6
avg_dist = 0.4458689458689459
dist = 0.24715099715099714
check = 0
avg_dist = 0.24715099715099714


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.25427350427350426
check = 2
avg_dist = 0.44515669515669515
dist = 0.24857549857549857
check = 0
avg_dist = 0.24857549857549857


10-element Vector{Float64}:
 0.24287749287749288
 0.24786324786324787
 0.25427350427350426
 0.26495726495726496
 0.2378917378917379
 0.26851851851851855
 0.245014245014245
 0.24715099715099714
 0.25427350427350426
 0.24857549857549857

In [53]:
dev = sqrt(sum(dist.^2)/length(dist)-(sum(dist)/length(dist))^2)/sqrt(length(dist))
1-M/N, sum(dist)/length(dist)

(0.24002849002849003, 0.2511396011396011)

In [31]:
1-M/N, 0.318686262747450

(0.24002849002849003, 0.31868626274745)

In [102]:
sum(H*(fg.fields .< 0) .% 2)

1

In [70]:
cw = fg.fields .< 0;
dist = sum(cw .!= y)/N

0.2167393675027263

In [71]:
1-M/N, dist

(0.30552526354053067, 0.2167393675027263)

In [707]:
B,indep=findbasis(H)

(
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣛⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⠛⣽⣿⣷⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⢬⢿⣿⣿⢿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇, [1305, 5928, 239, 2177, 2836, 2337, 3619, 6060, 943, 9471  …  1620, 10099, 3460, 10249, 7536, 10732, 3862, 5283, 2205, 6647])

In [708]:
B

11004×3362 SparseMatrixCSC{Int64,Int64} with 71353 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣛⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⠛⣽⣿⣷⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⢬⢿⣿⣿⢿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇

In [233]:
H .= 0

13469×17004 SparseMatrixCSC{Int64,Int64} with 40406 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⢽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠿⠿⠿⠿⠻⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿

In [540]:
isuppertriang(Hnew)

true

In [506]:
findbasis(H)

LoadError: AssertionError: isuppertriang(T)

In [521]:
    rowperm, colperm = leaf_removal(H)
    Hnew = H[rowperm, colperm]


7642×11004 SparseMatrixCSC{Int64,Int64} with 22925 stored entries:
⠙⢾⣬⣐⢄⡖⢐⡌⣄⢌⡥⢊⢹⡀⣒⢂⠀⠘⠈⠀⢒⣰⢠⢊⢘⣎⠕⣸⡰⣀⡔⣢⠠⠥⠘⡀⢒⡲⡀⡠
⠀⠀⠙⢮⡓⠦⣇⠀⠁⣦⠨⣕⠨⢞⡏⢈⣤⣄⢅⠀⠼⡣⡇⠀⠰⢊⠋⢼⡄⠄⡄⠘⣳⢋⠇⠺⢤⠡⠄⣉
⠀⠀⠀⠀⠙⢦⡈⠙⠾⣦⢨⡤⠨⠮⢈⠗⣊⢁⢁⡐⣣⠦⢐⢲⡥⠌⣋⠺⣧⠘⡄⣪⠐⢕⠋⠦⡠⠄⠫⡤
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠙⠶⣥⢪⠆⠠⡝⢒⠱⠅⣨⡲⠠⠈⢏⠉⣤⠁⣿⣯⣍⠍⢄⣺⣄⠪⡃⣁⠔⡂
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠙⢧⣆⣐⡯⣼⡪⡴⠛⢒⡼⠂⡟⡠⠆⢻⣷⣮⡛⡄⠕⠈⢇⠑⠴⡆⢇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠳⣾⢄⢊⡘⢱⡚⡥⣠⣉⢽⠁⠸⣿⣿⣆⢒⢆⢿⡕⠧⡵⡢⢦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠙⢧⡍⡲⠮⡊⠂⣳⣱⠩⠀⢿⣿⣿⣮⠱⡊⡎⠨⡀⡡⠈
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⢷⣵⢵⢓⢚⠓⣇⠀⠈⣿⣿⣿⣯⠘⠶⠕⠅⢤⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⣇⠜⡱⠐⣂⠀⠀⠘⣿⣿⣿⣿⣥⠧⡮⢜⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⢻⣥⣜⡆⠀⠀⠀⠘⢿⣿⣿⣿⣝⣱⢼⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⡼⠧⠀⠀⠀⠀⠈⢿⣿⣿⣿⣶⣅⣲
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠹⡖⠀⠀⠀⠀⠀⠀⠙⣿⣿⣿⣧⠼
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⢯⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠾⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠿

In [525]:
isuppertriang(Hnew)

false

In [523]:
dropzeros!(Hnew)

7642×11004 SparseMatrixCSC{Int64,Int64} with 22925 stored entries:
⠙⢾⣬⣐⢄⡖⢐⡌⣄⢌⡥⢊⢹⡀⣒⢂⠀⠘⠈⠀⢒⣰⢠⢊⢘⣎⠕⣸⡰⣀⡔⣢⠠⠥⠘⡀⢒⡲⡀⡠
⠀⠀⠙⢮⡓⠦⣇⠀⠁⣦⠨⣕⠨⢞⡏⢈⣤⣄⢅⠀⠼⡣⡇⠀⠰⢊⠋⢼⡄⠄⡄⠘⣳⢋⠇⠺⢤⠡⠄⣉
⠀⠀⠀⠀⠙⢦⡈⠙⠾⣦⢨⡤⠨⠮⢈⠗⣊⢁⢁⡐⣣⠦⢐⢲⡥⠌⣋⠺⣧⠘⡄⣪⠐⢕⠋⠦⡠⠄⠫⡤
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠙⠶⣥⢪⠆⠠⡝⢒⠱⠅⣨⡲⠠⠈⢏⠉⣤⠁⣿⣯⣍⠍⢄⣺⣄⠪⡃⣁⠔⡂
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠙⢧⣆⣐⡯⣼⡪⡴⠛⢒⡼⠂⡟⡠⠆⢻⣷⣮⡛⡄⠕⠈⢇⠑⠴⡆⢇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠳⣾⢄⢊⡘⢱⡚⡥⣠⣉⢽⠁⠸⣿⣿⣆⢒⢆⢿⡕⠧⡵⡢⢦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠙⢧⡍⡲⠮⡊⠂⣳⣱⠩⠀⢿⣿⣿⣮⠱⡊⡎⠨⡀⡡⠈
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⢷⣵⢵⢓⢚⠓⣇⠀⠈⣿⣿⣿⣯⠘⠶⠕⠅⢤⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⣇⠜⡱⠐⣂⠀⠀⠘⣿⣿⣿⣿⣥⠧⡮⢜⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⢻⣥⣜⡆⠀⠀⠀⠘⢿⣿⣿⣿⣝⣱⢼⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⡼⠧⠀⠀⠀⠀⠈⢿⣿⣿⣿⣶⣅⣲
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠹⡖⠀⠀⠀⠀⠀⠀⠙⣿⣿⣿⣧⠼
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⢯⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠾⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠿

In [531]:
Hnew[:,1:size(Hnew,1)]

7642×7642 SparseMatrixCSC{Int64,Int64} with 15740 stored entries:
⠙⢷⢮⣐⠐⠀⡀⡐⠂⠔⡐⡀⠀⠠⢉⡀⠒⠈⡆⠀⠠⠄⠂⠀⠀⠌⠐⠀⠀⠔⠀⢐⢀⠠⠁⠸⡂⠡⠅⢀
⠀⠀⠑⢌⠙⠦⣔⡂⠀⡍⠈⠢⢁⠀⠋⠰⢐⠀⠊⡡⡬⠆⠅⠀⠀⠐⠀⠀⠀⠆⠩⠐⡁⠈⠀⠐⡪⢡⠰⠐
⠀⠀⠀⠀⠑⢄⠀⠙⠲⣅⡀⠀⢀⠳⠂⡂⡆⠀⠒⠨⠆⠀⡠⢒⣈⣀⠂⠄⠈⡈⢋⠱⠃⠀⠀⠑⠠⡀⡀⠡
⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠙⠲⣧⡠⠀⡄⡠⠀⣘⠪⠀⠩⡠⠂⠁⠀⢀⠀⠌⠂⢴⠀⠐⢐⢌⠠⡀⠐⠠⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⠲⣬⣅⠂⠀⠤⢀⠁⠀⠮⡀⡁⢄⠨⠐⠐⠄⡀⠁⠈⢔⠑⠠⠔⠃⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠙⠺⣅⠘⠈⠀⠐⠔⠀⣀⠈⡈⢀⠪⠡⠐⠘⠀⡐⠤⣀⠜⢁⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠈⠙⢧⣐⡀⠐⡉⠠⡠⠆⣀⡰⠍⠁⢇⣔⠆⠂⡓⠀⠠⠆
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠈⠳⣍⣨⡐⠀⢡⠐⠰⠤⡄⡤⠂⠀⠐⠈⠊⠰⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⢦⣂⢄⠴⢀⠈⠐⡂⠊⡃⠶⠲⠐⠅⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⠳⣄⠅⢙⠰⠰⣐⠈⠀⠱⠈⣃⠅
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠈⠳⣪⣐⢄⠠⣠⠄⡉⠵⠔⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⢮⡌⠵⠁⢴⠀⠄⠀⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⢦⡀⠌⡐⠁⠉⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠳⡎⠐⡀⢆⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠙⣮⠪⡜⠅
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠘⣗⠓⠤
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠘⣦⠴
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠘⡧
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⢳
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢼

In [536]:
for i=1:size(Hnew,1)
    if rowvals(Hnew)[last(nzrange(Hnew,i))] > i
        println(i, " ", nzrange(Hnew,i))
    end
end

In [533]:
Hnew[70:80,70:80]

11×11 SparseMatrixCSC{Int64,Int64} with 11 stored entries:
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1

In [519]:
nzrange(Hnew,1)

1:1

In [617]:
f(x)=(i=1;println(x))

f (generic function with 1 method)

In [619]:
f("ciao $i")

LoadError: UndefVarError: i not defined

In [148]:
?GaloisField

search: GaloisField GaloisFields @GaloisField @GaloisField!



```
const F = GaloisField(p)
const F,α = GaloisField(p, :β => [1, 0, 1])
const F,α = GaloisField(p, n, :β)
```

Return a type representing a finite field.

The single-argument signature returns the finite field $ℤ/pℤ$.

The two-arguments signature returns an algebraic extension of that field, with minimum polynomial given by the second argument: a dense representation of the univariate, monic polynomial, with ascending degree.

The three-arguments signature returns an algebraic extension of that field, with minimum polynomial equal to the [Conway polynomial](https://en.wikipedia.org/wiki/Conway_polynomial_(finite_fields))  for $(p,n)$. The `GaloisFields` package ships with a database of Conway  polynomials and will raise an error if it does not contain an entry for  $(p,n)$.

Note that in the latter two cases, the variable name (e.g. β above) is part of the type. This lets you define identifications between isomorphic (sub)fields. For example, with the following definition

```
const F = @GaloisField! 𝔽₂ β^2 + β + 1
const G = @GaloisField! 𝔽₂ γ^2 + γ + 1
```

the fields $F$ and $G$ are isomorphic, but not canonically. We might define

```
@GaloisFields.identify β => γ + 1
@GaloisFields.identify γ => β + 1
```

to allow for conversions like

```
G(β)
convert(F, γ + 1)
```

In the Conway case, you do not have to define your own identifications, as the Conway polynomials satisfy compatibility relations that allow us to use certain distinguished inclusions between them.


In [335]:
x=[rand(2) for i=1:10]

10-element Vector{Vector{Float64}}:
 [0.7248512048074602, 0.5956281398569836]
 [0.3423616342729472, 0.30990956468641717]
 [0.2220817188124009, 0.0919372195820698]
 [0.43783398880046787, 0.47016092384227326]
 [0.791725980123458, 0.7052410433796314]
 [0.09217476498061083, 0.12725939866504188]
 [0.45281383084093174, 0.39122464103589816]
 [0.33806263297107253, 0.9542242098549534]
 [0.5664683799053358, 0.4337947502526993]
 [0.04083282329424409, 0.40240658284532693]

In [344]:
x .+= tuple([1,1])

10-element Vector{Vector{Float64}}:
 [5.72485120480746, 5.595628139856983]
 [5.342361634272947, 5.309909564686417]
 [5.222081718812401, 5.09193721958207]
 [5.437833988800468, 5.470160923842274]
 [5.7917259801234575, 5.7052410433796314]
 [5.09217476498061, 5.127259398665042]
 [5.452813830840932, 5.391224641035898]
 [5.338062632971073, 5.954224209854953]
 [5.566468379905336, 5.4337947502527]
 [5.040832823294244, 5.402406582845327]